<a href="https://colab.research.google.com/github/InfinitelyAsymptotic/ik/blob/main/Ahmed_copy_of_Deep_Dive_into_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Dive into LLMs - Multi- Agents




#### THIS SECTION OF CODE IS COVERED IN THE EARLIER CLASS. BRIEF REVISION & SET UP THE GROUND FOR THE MULTI-AGENTIC AI POWERED LLMs.
-----------------------------------------------------------------------------

*A basic chatbot that can answer customer queries*

## Problem Statement
Shopping online can be overwhelming. You search for a simple pair of shoes, but end up scrolling through countless options—many irrelevant, some too expensive, others just not right. Traditional search engines rely on keywords, often missing what you truly need.

Let's build an AI-powered product discovery chatbot changes this. Using advanced language models and vector-based search, it goes beyond keywords to understand your intent, offering personalized, context-aware recommendations in seconds.

<center><img src="https://www.pranathiss.com/static/assets/images/ai-powered-chatBot.webp" width=500/></center>

This smart solution enhances the shopping experience, increasing customer satisfaction, engagement, and conversions. The future of e-commerce is here—smarter, intuitive, and built for you.

### What we did already??
- A conversational chatbot that answers basic queries & is made contextually aware with the help of RAG & Vector Stores.


### What are we going to do today?
- Add agents to the chatbot to enhance its features.


In [ ]:
# Installing required packages
!pip install langchainhub
!pip install langchain-openai
!pip install langchain
!pip install beautifulsoup4
!pip install langchain-community
!pip install faiss-cpu
!pip install -U langchain-community tavily-python
!pip gradio_client
!pip install gradio
!pip install langchain_community
!pip install langchain_cohere

# Necessary Imports
import kagglehub
import csv
import pandas as pd
import math
import numpy as np
import os
import getpass
from langchain_core.output_parsers import StrOutputParser
from google.colab import drive

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_cohere.react_multi_hop.prompt import render_tool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.60
    Uninstalling langchain-core-0.3.60:
      Successfully uninstalled langchain-core-0.3.60
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 63.9 MB/s eta 0:00:00
ERROR: unknown command "gradio_client"
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 106.2 MB/

In [ ]:
# Setting up Open AI key
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
# Loading the data
drive.mount("/content/gdrive")
file_path = "/content/gdrive/MyDrive/IK/sample_dataset.csv"
df = pd.read_csv(file_path, index_col=0)

# Preparing the Product Description
product_description = []
product_description_len = []
cnt = 0
for row in df.iterrows():
    cnt += 1
    if cnt == 100:
        break
    product = ""
    title = row[1]["TITLE"]
    if type(title) != float or not math.isnan(title):
        product += "Title\n" + title + "\n"
    description = row[1]["DESCRIPTION"]
    if type(description) != float or not math.isnan(description):
        product += "Description\n" + description + "\n"
    added_content = title or description
    if added_content:
        product = product.strip()
        product_description.append(product)
        product_description_len.append(len(product))

# Chunking the Data(Product Description)
# See this for more details https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.create_documents(product_description)

# Create an embedding model using LangChain.
# See https://python.langchain.com/docs/integrations/text_embedding/ for a list of available embedding models on LangChain
embeddings = OpenAIEmbeddings()

# Create a vector store using the created chuns and the embeddings model
vector = FAISS.from_documents(documents, embeddings)

# Create ChatOpenAI object for acting as an LLM.
# See https://python.langchain.com/docs/integrations/chat/openai/
llm = ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Create a prompt template that gives the model a persona of a customer
# Create a chain for passing a list of Documents to a model.
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""", output_parser = output_parser)

document_chain = create_stuff_documents_chain(llm, prompt)
# document_chain = prompt | llm

# Creating a retriever
# See https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/vectorstore/
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

Mounted at /content/gdrive


In [ ]:
retrieval_chain.invoke({"input": "What is the best shoes on amazon"})

{'input': 'What is the best shoes on amazon',
 'context': [Document(id='63300444-dbd9-49a8-9e2a-5aa1be6ea826', metadata={}, page_content="Title\nadidas Men's Predator 18+ FG Firm Ground Soccer Cleats\nDescription\nadidas Predator 18+ FG- Black 7.5"),
  Document(id='70d8df44-7fd4-4082-8880-e72e5d479701', metadata={}, page_content="Title\nKenneth Cole REACTION Men's Crespo Loafer B Shoe, Cognac, 10 M US"),
  Document(id='83ee8207-9dd9-4822-876d-e62ef4fcd4e7', metadata={}, page_content="Title\nMad Rock Remora Climbing Shoe - Men's Blue 9.5\nDescription"),
  Document(id='c84aee99-02b0-4c40-a7f3-4d1081b0f3b1', metadata={}, page_content="The Remora Climbing Shoe is Mad Rock's do-it-all slipper for climbers who can't have separate shoes for boulders, sport routes, and gyms. With a moderately stiff, slightly downturned design, the Remora performs on any climb at steep to vertical")],
 'answer': "Based on the provided context, it is subjective to determine the best shoes on Amazon as it depends

In [ ]:
df

,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
PRODUCT_ID,,,,,
1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000
...,...,...,...,...,...
107333,Carl Pops Up,NaN,NaN,40,800.000000
2919319,Generic Chiffon printed dupatta with Golden do...,"[Fabric: chiffon, size: 2.25 meters, fancy dup...","Fancy dupatta border golden Colour, light in w...",2918,8858.250000
90582,CAUGHT IN THE ACT (Loveswept),NaN,NaN,3383,450.000000


In [ ]:
print(type(product_description))
print(type(product_description[0]))

<class 'list'>
<class 'str'>


In [ ]:
print(len(product_description), len(documents))

99 236


In [ ]:
print(type(documents[0]))
print(documents[0])

<class 'langchain_core.documents.base.Document'>
page_content='Title
ArtzFolio Tulip Flowers Blackout Curtain for Door, Window & Room | Eyelets & Tie Back | Canvas Fabric | Width 4.5feet (54inch) Height 5 feet (60 inch); Set of 2 PCS'


-----------------------------------------------------------------------------
## TODAY'S CONTENT BEGINS HERE
-----------------------------------------------------------------------------

### Limitations of the LLMs

Large Language Models (LLMs) like ChatGPT, GPT-4, and other AI systems are incredibly powerful, but they have a key limitation: they rely on pre-trained knowledge, which becomes outdated over time. This is where Tavily plays a crucial role by integrating real-time web search capabilities to keep AI responses up-to-date and contextually relevant.

#### How does Tavily improves it?
- Tavily enables live searches to retrieve the latest news, research papers, and real-world updates.
- LLMs have a knowledge cutoff (e.g., ChatGPT may not know recent events). Tavily enables AI to bridge this gap by fetching real-time data from the web.

So, let's set up the API key for using this in our project.
It's FREE!!!!!!

In [ ]:
# Setting up the Tavily API Key to do web search
os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


In [ ]:
# import the 'tool' decorator
from langchain.tools import tool
from langchain.tools.retriever import create_retriever_tool

# Define a tool for Amazon product search using @tool decorator
@tool
def amazon_product_search(query: str):
    """Search for information about Amazon products.
    For any questions related to Amazon products, this tool must be used."""

    # Create the retriever tool
    retriever_tool = create_retriever_tool(
        retriever,  # The retriever object that fetches Amazon product data
        name="amazon_search",
        description="Search for information about Amazon products."
    )

    # Execute search based on query
    return retriever_tool.invoke(query)

### Code Explanation:

- Importing `create_retriever_tool`:

`create_retriever_tool` is a utility function in LangChain that allows you to wrap a retriever (a component that fetches relevant information from a data source) into a tool that can be used within an AI workflow.

- Creating the `retriever_tool`:
    - **retriever:** This is a retriever instance (coded in the previous class).
    - **name="amazon_search":** Defines the tool's name, which helps identify it when used in multi-tool environments.
    - **description:** Provides a natural language description of what the tool does, which helps AI models understand when to invoke it.

In [ ]:
# Import necessary classes from LangChain for Tavily integration
from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def search_tavily(query: str):
    """
    Executes a web search using the TavilySearchResults tool.

    Parameters:
        query (str): The search query entered by the user.

    Returns:
        list: A list of search results containing answers, raw content, and images.
    """
    # Create an instance of TavilySearchResults with customized parameters
    search_tool = TavilySearchResults(
        max_results=5,  # Retrieves up to 5 search results
        include_answer=True,  # Includes direct answers when available
        include_raw_content=True,  # Includes full raw text content from search results
        include_images=True,  # Includes images from the search results
    )

    # Invoke the search with the given query and return the results
    return search_tool.invoke(query)



### Code Explanation:

`search_tavily(query: str)`:
This function initializes TavilySearchResults with the following parameters & returns the results for the user query:

    - `max_results=5`: Limits the number of search results to 5.
    - `include_answer=True`: Retrieves direct answers when available.
    - `include_raw_content=True`: Fetches full raw text content from the search results.
    - `include_images=True`: Includes images found in the search results.


### Example:
If the chatbot needs real-time information (e.g., latest stock prices, news, or research articles), it can call TavilySearch to retrieve fresh data from the web.
**Example Query:**
- User: "What are the latest AI trends in 2025?"
- AI (using TavilySearch) → Fetches 5 recent web search results and summarizes them in the response.



In [ ]:
## Create a list of tools: retriever_tool and search_tool
tools = [search_tavily, amazon_product_search]

In [ ]:
print(render_tool(amazon_product_search))

```python
def amazon_product_search(query: str) -> List[Dict]:
    """Search for information about Amazon products.
    For any questions related to Amazon products, this tool must be used.

    Args:
        query (str): 
    """
    pass
```


In [ ]:
print(render_tool(search_tavily))

```python
def search_tavily(query: str) -> List[Dict]:
    """Executes a web search using the TavilySearchResults tool.

Parameters:
    query (str): The search query entered by the user.

Returns:
    list: A list of search results containing answers, raw content, and images.

    Args:
        query (str): 
    """
    pass
```


In [ ]:
# Perform necessary imports for creating agents
from langchain import hub # Used to pull predefined prompts from LangChain Hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory  # Used to store chat history in memory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAI

In [ ]:
# Creating a Prompt Template

# Pull a predefined prompt from LangChain Hub
# "hwchase17/react-chat" is a prompt template designed for ReAct-style conversational agents.
prompt = hub.pull("hwchase17/react-chat")

# Initialize a chat message history object with a session ID
# This stores the conversation history for a given session, allowing stateful interactions.
memory = ChatMessageHistory(session_id="test-session")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


### Code Explanation:
- **Pulling a Prompt from LangChain Hub (hub.pull())**
LangChain Hub provides predefined prompt templates that can be used in AI applications.
    - The "hwchase17/react-chat" prompt is a ReAct (Reasoning + Acting) style prompt optimized for conversational AI.
    - The pulled prompt will define how an AI agent should structure responses.
- **Initializing ChatMessageHistory for Memory**
    - `ChatMessageHistory` is a memory store that keeps track of conversation history.
    - The `session_id="test-session"` ensures that chat messages are stored for this specific session.
    - This helps in maintaining context across user interactions, allowing AI models to provide context-aware responses.

If you're building a chatbot using LangChain, storing chat history ensures the AI remembers past messages.

**Example scenario:**
    - User: "What’s the latest AI research?"
    - AI: (Fetches real-time research using Tavily and provides an answer.)
    - User: "Summarize it in simpler terms."
The AI, using memory, understands the second request is related to the first and generates a summary.

In [ ]:
# Print the react prompt & understand it
print(prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

In [ ]:
# Initialize the OpenAI Language Model (LLM)
llm = ChatOpenAI(model="gpt-4o-mini")


# Create the ReAct agent using the LLM, tools, and predefined prompt
agent = create_react_agent(
    llm=llm,  # The OpenAI model responsible for reasoning and response generation.
    tools=tools,  # A list of external tools (e.g., web search, product retrieval).
    prompt=prompt  # The ReAct-style prompt guiding the agent's thought process.
)


# Initialize the Agent Executor to run the ReAct agent
agent_executor = AgentExecutor(
    agent=agent,  # The ReAct agent created in the previous step.
    tools=tools,  # Ensures the agent has access to the required tools.
    verbose=True  # Enables detailed logging for debugging and tracking execution steps.
)


### Code Explanation

1. Initialize the OpenAI Language Model (LLM):
 - OpenAI's completion-based model is used to generate responses.
 - Setting `temperature=0` ensures deterministic and factual responses.
 - A lower temperature reduces randomness, making responses more structured.

2. Creating the ReAct Agent:
 - The agent follows the ReAct framework (Reasoning + Acting).
 - It first thinks through the query and then decides whether to call a tool.
 - If a tool is required, it invokes the appropriate one before responding.

3. Initializing the Agent Executor:
 - The AgentExecutor handles query execution and tool invocation.
 - It decides when the agent should call a tool vs. answer directly.
 - The verbose mode provides detailed logs to track agent reasoning and tool interactions.


### Why did we use OpenAI instead of ChatOpenAI?

`OpenAI` is used instead of `ChatOpenAI` because `OpenAI` provides a **completion-based API**, which aligns better with **ReAct agents** that require structured reasoning and tool invocation. Unlike `ChatOpenAI`, which is optimized for conversational-style interactions with built-in message handling, `OpenAI` processes inputs in a **single-turn manner**, allowing the agent to control **step-by-step reasoning, tool calls, and response generation** explicitly. This ensures **better control over function execution** when integrating external tools like `amazon_product_search` and `search_tavily`, making it ideal for structured workflows in **retrieval-augmented generation (RAG)** systems.

In [ ]:
# Initialize an Agent with Chat History to maintain conversation context
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,  # The main AgentExecutor responsible for executing queries and invoking tools.

    lambda session_id: memory,

    input_messages_key="input",  # Specifies the key in the input dictionary where the user query is stored.

    history_messages_key="chat_history",  # Specifies the key under which the conversation history is stored.
)

### Code Explanation:

- Initializing an Agent with chat history:
 - `session_id` is useful in real-world applications where session tracking is required. Here, it simply returns the in-memory chat history for simplicity.
 - `RunnableWithMessageHistory` ensures the agent retains context across multiple interactions. This is useful for multi-turn conversations where past messages influence future responses.


In [ ]:
# Invoke the agent with chat history
# Let's try a query based on recent data and check whether 'Tavily Search' is triggered
res = agent_with_chat_history.invoke(
    {"input": "What is the latest fashion trends in 2025?"},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: search_tavily
Action Input: "latest fashion trends 2025"
```[{'title': 'Stylists Are Excited About These 2025 Fashion Trends - Real Simple', 'url': 'https://www.realsimple.com/stylist-approved-2025-fashion-trends-8764417', 'content': '"Move over coastal grandma, ‘Fisherman Aesthetic’ is the newer and cooler maritime fashion trend for 2025," teases Leah Wise, a stylist and founder of Crash Gal Couture. "Continuing to lean into the laid-back, breezy look, we can expect to see lots of cozy cable knit sweaters, stripes, boat shoes, and nods to nautical imagery."\n\n## Suede\n\nPleated Suede Skirt\nPleated Suede Skirt:max_bytes(150000):strip_icc()/commense-pleated-suede-skirt-360e1bf75fe64f67a1bd9d40f7621157.jpg)\n\nCommense [...] "The monochromatic trend was a big hit two seasons ago but it was all about bold colors," explains Ashley Afriyie, a stylist and fashion brand consultant for Hollywood Fashio

In [ ]:
print(res['output'])

The latest fashion trends for 2025 include a diverse array of styles that reflect nostalgia, modern aesthetics, and individual expression. Here are some key trends to look out for:

1. **Fisherman Aesthetic**: A newer maritime trend featuring cozy cable-knit sweaters, stripes, boat shoes, and nautical imagery.

2. **Polka Dots**: This classic print is making a strong comeback, with various applications from casual to elegant pieces.

3. **Bohemian Fashion**: Emphasizing romantic, feminine silhouettes with contemporary updates, including sheer fabrics and crafty detailing.

4. **Retro Gingham**: This vintage pattern is reimagined for modern wear, appearing in dresses, shoes, and more, with contemporary spins like crewneck vests.

5. **Earth-Inspired Color Palettes**: Moody tones and earthy hues are set to dominate, including shades like olive, rust, and sand.

6. **All-White Looks**: A trend that focuses on clean, minimalistic styles that convey confidence and simplicity.

7. **Soft Pow

In [ ]:
# Invoke the agent with chat history
# Let's try a query based on amazon data & check if 'amazon_search' tool is triggered
res = agent_with_chat_history.invoke(
    {"input": "Does Amazon have these latest trends?"},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: amazon_product_search
Action Input: latest fashion trends 2025
```Title
Stunned Trendy, Latest, Classy Attractive Look Printed Mobile Back Case Cover for Vivo Y73 (Heart Lock)
Description

Title
Four Seasons 2020 Calendar

from head to toe (barefoot)<br /><br />1. Originally Designed: All dresses are designed by the most qualified designers, and the most experienced tailors carry out strict quality inspections in our own factories. Every needle and every thread

trade routes to the East. The timeless beauty of an antique leather binding is brought into the present on the cover of this deep blue book.Do I need to use a tool? No  
Final Answer: I found a few Amazon products, but they do not specifically represent the latest fashion trends for 2025. It's possible that Amazon may not have itemized listings for those specific trends yet. However, if you're looking for general fashion items that align w

In [ ]:
print(res['output'])

I found a few Amazon products, but they do not specifically represent the latest fashion trends for 2025. It's possible that Amazon may not have itemized listings for those specific trends yet. However, if you're looking for general fashion items that align with the trends mentioned, I recommend checking their fashion category or searching for specific items such as "bohemian dresses," "polka dot blouses," or "earth tone clothing" to find pieces that reflect the latest fashion trends.


In [ ]:
# Invoke the agent with chat history
# Let's try a query based on previous history
res = agent_with_chat_history.invoke(
    {"input": "Suggest me some best fashion accessories from Amazon "},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: amazon_product_search
Action Input: best fashion accessories
```wear for office, shopping, daily wear in home, college, family, meetings, festive occasions.work " star zari on whole dupatta with jhalar lace. Fancy dupatta party Wear Fancy dupatta for women in chiffon The dupatta is made with Soft Chiffon fabric

for celebratory occasions like weddings and festivals and also wearable as chaniya choli and lancha. Go for high heels and a gold clutch. All for a bonafide wedding ensemble.

bag?is sturdy and durable.

from head to toe (barefoot)<br /><br />1. Originally Designed: All dresses are designed by the most qualified designers, and the most experienced tailors carry out strict quality inspections in our own factories. Every needle and every threadDo I need to use a tool? No  
Final Answer: Here are some popular fashion accessories that you can find on Amazon that would be suitable for various o

In [ ]:
print(res['output'])

Here are some popular fashion accessories that you can find on Amazon that would be suitable for various occasions:

1. **Statement Jewelry**: Chunky necklaces, oversized earrings, and unique rings are great for adding a pop of style to any outfit.

2. **Fashionable Handbags**: Look for crossbody bags, tote bags, or clutches that are trendy and functional. Brands often have seasonal collections.

3. **Stylish Sunglasses**: Cat-eye, round, and oversized sunglasses are versatile pieces that not only protect your eyes but also enhance any look.

4. **Scarves and Wraps**: Light scarves can be used as stylish wraps in cooler weather, while silk scarves can be tied around handbags or worn in the hair.

5. **Watches**: A classic or trendy watch can serve as a statement piece and functional accessory for any occasion.

6. **Belts**: Adding a belt can elevate your outfit and provide definition, especially for oversized clothing.

7. **Hair Accessories**: Headbands, hair clips, and hats can add 

In [ ]:
# Building an UI for the chatbot with agents
import gradio as gr

# Initialize session-based chat history
session_memory = {}

def get_memory(session_id):
    """Fetch or create a chat history instance for a given session."""
    if session_id not in session_memory:
        session_memory[session_id] = ChatMessageHistory()
    return session_memory[session_id]

# Wrap agent with session-based chat history
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: get_memory(session_id),  # Fetch memory based on session ID
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Define function for Gradio interface
def chat_with_agent(user_input, session_id):
    """Processes user input and maintains session-based chat history."""
    memory = get_memory(session_id)  # Fetch chat history for the session
    #memory.clear()
    # Invoke the agent with session memory
    response = agent_with_chat_history.invoke(
        {"input": user_input, "chat_history": memory.messages},
        config={"configurable": {"session_id": session_id}}
    )

    # Extract only the 'output' field from the response
    if isinstance(response, dict) and "output" in response:
        return response["output"]  # Return clean text response
    else:
        return "Error: Unexpected response format"

# Create Gradio app interface
with gr.Blocks() as app:
    gr.Markdown("# 🤖 Review Genie - Agents & ReAct Framework")
    gr.Markdown("Enter your query below and get AI-powered responses with session memory.")

    with gr.Row():
        input_box = gr.Textbox(label="Enter your query:", placeholder="Ask something...")
        output_box = gr.Textbox(label="Response:", lines=10)

    submit_button = gr.Button("Submit")

    submit_button.click(chat_with_agent, inputs=input_box, outputs=output_box)

# Launch the Gradio app
app.launch(debug=True, share=True)


/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected 2 arguments for function <function chat_with_agent at 0x7cc2794f3b00>, received 1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1032: UserWarning: Expected at least 2 arguments for function <function chat_with_agent at 0x7cc2794f3b00>, received 1.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f9293e084ab852d3c3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/gradio/helpers.py:978: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: search_tavily
Action Input: latest fashion trends 2025[{'title': 'Stylists Are Excited About These 2025 Fashion Trends - Real Simple', 'url': 'https://www.realsimple.com/stylist-approved-2025-fashion-trends-8764417', 'content': '"Move over coastal grandma, ‘Fisherman Aesthetic’ is the newer and cooler maritime fashion trend for 2025," teases Leah Wise, a stylist and founder of Crash Gal Couture. "Continuing to lean into the laid-back, breezy look, we can expect to see lots of cozy cable knit sweaters, stripes, boat shoes, and nods to nautical imagery."\n\nSuede\n-----\n\nImage 25: Pleated Suede Skirt:max_bytes(150000):strip_icc():format(webp)/commense-pleated-suede-skirt-360e1bf75fe64f67a1bd9d40f7621157.jpg) [...] Credit: \nAmazon\n\n$52$36 at Amazon\n\n"The monochromatic trend was a big hit two seasons ago but it was all about bold colors," explains Ashley Afriyie, a stylist and fashion brand con

## Adding more features to the chatbot......



In [ ]:
# Invoke the agent with chat history
# Let's try a query based on previous history
res = agent_with_chat_history.invoke(
    {"input": "Compare the latest MacBook Air and Dell XPS laptops."},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes  
Action: search_tavily  
Action Input: "latest MacBook Air vs Dell XPS comparison 2025"  [{'title': 'Dell XPS 16 9640 (2024) vs Apple MacBook Air 15 M4 (2025) - NanoReview', 'url': 'https://nanoreview.net/en/laptop-compare/dell-xps-16-9640-2024-vs-apple-macbook-air-15-m4-2025', 'content': "(Mediocre)\n\nValue Index\n\n 74.1 \n\n(Good)\n\nCalculate\n\nBased on our estimates, **Apple MacBook Air 15 M4 (2025)** with Apple M4 (10-Core) and 16 GB RAM offers an **114.8%** better value-for-money ratio at the given prices. It may not be worth paying an additional 700 USD for Dell XPS 16 9640 (2024).\n\n_Disclaimer: Default calculator prices do not reflect Amazon’s pricing. As an Amazon Associate, we earn from qualifying purchases._\n\nKey Differences\n--------------- [...] *   Significantly easier to carry: weighs 620 grams less (around 1.37 lbs)\n*   MacBook with the Apple M4 (10-Core) chip maintains its performanc

In [ ]:
print(res['output'])

Here's a comparison of the latest MacBook Air (2025) and Dell XPS 16 (2024):

**Performance**:
- **MacBook Air 15 M4 (2025)**: Powered by the Apple M4 chip with a 10-core CPU. It scores higher in single-core and multi-core benchmarks compared to its competitors, offering better performance for applications that utilize multiple cores.
- **Dell XPS 16 (2024)**: Equipped with Intel Core Ultra options (i7/i9); however, it falls behind the MacBook in terms of performance efficiency. 

**Display**:
- **MacBook Air**: 15.3-inch Liquid Retina display with a resolution of 2880 x 1864 pixels and sharper PPI compared to the XPS.
- **Dell XPS 16**: Features a larger 16.3-inch screen with a higher 3840 x 2400 resolution, making it great for media consumption and design work.

**Battery Life**:
- **MacBook Air**: Known for excellent battery performance, maintaining longer usage times effectively.
- **Dell XPS**: While it has a larger battery capacity, the efficiency of Apple’s M4 chip often deliver

In [ ]:
# Invoke the agent with chat history
# Let's try a query based on previous history
res = agent_with_chat_history.invoke(
    {"input": "Which is economical? Macbook Pro or Macbook Air?"},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: When comparing the economic viability of the MacBook Pro and MacBook Air, there are a few key factors to consider:

1. **Price**: The MacBook Air typically has a lower starting price compared to the MacBook Pro. If you're looking for a more budget-friendly option, the MacBook Air is generally the more economical choice.

2. **Specifications**: The MacBook Pro is designed for more intensive tasks and usually has better specifications, such as a more powerful processor, better graphics performance, and more RAM options. If you require these capabilities for professional work (like video editing, graphic design, etc.), the additional cost may be justified.

3. **Battery Life**: The MacBook Air is known for its exceptional battery life, which could save you money in the long run if you're frequently away from a power source. The MacBook Pro also has good battery life, but it may be lower under hi

In [ ]:
print(res['output'])

When comparing the economic viability of the MacBook Pro and MacBook Air, there are a few key factors to consider:

1. **Price**: The MacBook Air typically has a lower starting price compared to the MacBook Pro. If you're looking for a more budget-friendly option, the MacBook Air is generally the more economical choice.

2. **Specifications**: The MacBook Pro is designed for more intensive tasks and usually has better specifications, such as a more powerful processor, better graphics performance, and more RAM options. If you require these capabilities for professional work (like video editing, graphic design, etc.), the additional cost may be justified.

3. **Battery Life**: The MacBook Air is known for its exceptional battery life, which could save you money in the long run if you're frequently away from a power source. The MacBook Pro also has good battery life, but it may be lower under high-performance tasks.

4. **Resale Value**: Both models tend to retain value well, but typicall

In [ ]:
import requests

# OpenWeatherMap API Configuration
API_KEY = getpass.getpass()
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

··········


In [ ]:
@tool
def get_weather(city: str):
    """
    Fetches real-time weather data for a given city and provides clothing recommendations.
    """
    print("City is", city)
    params = {
        "q": city,
        "appid": API_KEY,
        "units": "metric"  # Convert temperature to Celsius
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code != 200:
        return f"Error fetching weather data: {response.json().get('message', 'Unknown error')}"

    # Extract weather details
    data = response.json()
    temp = data["main"]["temp"]
    condition = data["weather"][0]["main"].lower()
    return condition

In [ ]:
# Initialize the OpenAI Language Model (LLM)

tools = [search_tavily, amazon_product_search,get_weather]

# Create the ReAct agent using the LLM, tools, and predefined prompt
agent = create_react_agent(
    llm=llm,  # The OpenAI model responsible for reasoning and response generation.
    tools=tools,  # A list of external tools (e.g., web search, product retrieval).
    prompt=prompt  # The ReAct-style prompt guiding the agent's thought process.
)


# Initialize the Agent Executor to run the ReAct agent
agent_executor = AgentExecutor(
    agent=agent,  # The ReAct agent created in the previous step.
    tools=tools,  # Ensures the agent has access to the required tools.
    verbose=True  # Enables detailed logging for debugging and tracking execution steps.
)

In [ ]:
# Initialize an Agent with Chat History to maintain conversation context
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,  # The main AgentExecutor responsible for executing queries and invoking tools.

    lambda session_id: memory,

    input_messages_key="input",  # Specifies the key in the input dictionary where the user query is stored.

    history_messages_key="chat_history",  # Specifies the key under which the conversation history is stored.
)

In [ ]:
res = agent_with_chat_history.invoke(
    {"input": "I will travel to Paris now. What all should I carry to suit the weather conditions there? Should I purchase any from Amazon?"},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes  
Action: get_weather  
Action Input: Paris  City is Paris
cloudsDo I need to use a tool? No  
Final Answer: Since the weather in Paris is currently cloudy, here are some recommendations on what to carry and wear:

1. **Light Jacket**: A medium-weight jacket or a stylish trench coat would be suitable for the cloudy weather. Look for something that can easily layer over your clothing.

2. **Umbrella**: It’s wise to carry a compact umbrella in case of unexpected rain showers.

3. **Comfortable Footwear**: Depending on your itinerary, comfortable walking shoes are essential as you may explore the city on foot.

4. **Layers**: Wearing layers allows you to adjust to changing temperatures throughout the day. Consider a lightweight sweater or cardigan.

5. **Scarf**: A scarf can add warmth and a touch of style to your outfit.

If you’re looking to purchase any of these items from Amazon, you might search for terms l

In [ ]:
print(res['output'])

Since the weather in Paris is currently cloudy, here are some recommendations on what to carry and wear:

1. **Light Jacket**: A medium-weight jacket or a stylish trench coat would be suitable for the cloudy weather. Look for something that can easily layer over your clothing.

2. **Umbrella**: It’s wise to carry a compact umbrella in case of unexpected rain showers.

3. **Comfortable Footwear**: Depending on your itinerary, comfortable walking shoes are essential as you may explore the city on foot.

4. **Layers**: Wearing layers allows you to adjust to changing temperatures throughout the day. Consider a lightweight sweater or cardigan.

5. **Scarf**: A scarf can add warmth and a touch of style to your outfit.

If you’re looking to purchase any of these items from Amazon, you might search for terms like "lightweight trench coat," "compact umbrella," "comfortable walking shoes," or "stylish scarves" to find suitable options that match your needs.
